# 语言模型

In [1]:
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
dataset = Dataset.load_from_disk("../../datas/wiki_cn_filtered")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("models/bloom-389")